In [ ]:
import json

import numpy as np
import pandas as pd
import plotly.graph_objs as go
from dash import Dash, Input, Output, callback, dcc, html
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from shared.spark_config import create_spark_config

from libs.configuration import configure

rng = np.random.default_rng(12345)
env = configure(["../.env.development", "../.env.development.local"])
conf = create_spark_config("M4_Presentation")

In [ ]:
spark = SparkSession.builder.config(conf=conf).getOrCreate()

In [ ]:
df_airports = spark.sql("SELECT icao, name, lat, lon FROM dev.tier1.airports").toPandas()

In [ ]:
# TODO
df_flights = (
    spark.table("dev.tier1.flight_signals")
    .select("icao24", "lat", "lon", "heading")
    .where(F.col("time"))
    .toPandas()
)

In [ ]:
layout = go.Layout(
    margin={"l": 0, "t": 0, "b": 0, "r": 0},
    mapbox=dict(
        style="outdoors",
        zoom=1,
        uirevision=True,
        accesstoken="pk.eyJ1IjoiYmx1bmRlcmVyODQ0OCIsImEiOiJjbHJ4YnhucHQxNDJsMmxwY3R6NDg2c3IzIn0.CALLf90eS4adTscrk5MqEQ",
    ),
    height=800,
)

In [ ]:
bounding_box = ()
df_airports_bounded = None
df_flights_bounded = None


def update_bounded_data(new_bounding_box: tuple):
    """@param new_bounding_box = (lat_min, lat_max, lon_min, lon_max)"""
    global bounding_box, df_airports_bounded

    if new_bounding_box != bounding_box:
        bounding_box = new_bounding_box
        lat_min, lat_max, lon_min, lon_max = bounding_box

        df_airports_bounded = df_airports.where(
            (df_airports.lat >= lat_min)
            & (df_airports.lat <= lat_max)
            & (df_airports.lon >= lon_min)
            & (df_airports.lon <= lon_max)
        )

        # df_flights = (
        #     spark.table("dev.tier1.flight_signals")
        #     .select("icao24", "lat", "lon", "heading")
        #     .where(F.col(""))
        #     .toPandas()
        # )

In [ ]:
app = Dash("M4_Presentation")
app.layout = html.Div(
    html.Div(
        [
            html.Pre(id="log"),
            dcc.Graph(id="liveupdate-map"),
            dcc.Interval(id="interval-component", interval=5 * 1000, n_intervals=0),
        ]
    )
)


@callback(
    Output("liveupdate-map", "figure"),
    (Input("interval-component", "n_intervals"), Input("liveupdate-map", "relayoutData")),
)
def update_graph_live(n: int, layout_data: dict):
    global df_airports_bounded

    figure = go.Figure(
        data=go.Scattermapbox(),
        layout=layout,
    )

    if layout_data is not None and "mapbox._derived" in layout_data:
        coords = layout_data["mapbox._derived"]["coordinates"]
        update_bounded_data((coords[2][1], coords[0][1], coords[0][0], coords[1][0]))

        figure = go.Figure(
            data=go.Scattermapbox(
                mode="markers",
                lon=df_airports_bounded.lon,
                lat=df_airports_bounded.lat,
                hoverinfo="text",
                hovertext=df_airports_bounded.name,
                marker=go.scattermapbox.Marker(
                    symbol="bar",
                    size=10,
                    color="#930093",
                    # opacity=0.5,
                ),
            ),
            layout=layout,
        )

    return figure


app.run(debug=True, jupyter_height=1000)